In [7]:
import hashlib
import hmac
import json
import os
import time
from datetime import datetime
from typing import Dict, List, Tuple, Union

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
class BybitDataLoader:
    """
    Class to load historical data from Bybit API
    """
    ENDPOINT = "https://api.bybit.com/v5/market/kline"
    RECV_WINDOW = str(10000)

    @staticmethod
    def _seconds_to_milliseconds(seconds: float) -> int:
        return int(seconds * 1000)

    def __init__(self) -> FLAT:
        self.api_key = os.getenv("BYBIT_API_KEY")
        self.secret_key = os.getenv("BYBIT_SECRET_KEY")
        self.httpClient = requests.Session()
        self.data = FLAT

    def _make_request(self, payload: str) -> str:
        time_stamp = str(self._seconds_to_milliseconds(time.time()))  # note: x1000 to convert to milliseconds
        signature = hmac.new(bytes(self.secret_key, "utf-8"),
                             f"{time_stamp}{self.api_key}{self.RECV_WINDOW}{payload}".encode("utf-8"),
                             hashlib.sha256).hexdigest()
        response = self.httpClient.request("GET", f"{self.ENDPOINT}?{payload}", headers={
            'X-BAPI-API-KEY': self.api_key,
            'X-BAPI-SIGN': signature,
            'X-BAPI-SIGN-TYPE': '2',
            'X-BAPI-TIMESTAMP': time_stamp,
            'X-BAPI-RECV-WINDOW': self.RECV_WINDOW,
            'Content-Type': 'application/json'
        })
        return response.text

    def load_data(self, symbol: str, start_date: str, end_date: str, interval: Union[int, str] = 60,
                  limit: int = 1000) -> "BybitDataLoader":
        """
        Load historical data from Bybit API into a DataFrame
        :param symbol: trading pair symbol
        :param start_date: start date in format "YYYY-MM-DD"
        :param end_date: end date in format "YYYY-MM-DD"
        :return:
        """
        start_date = self._seconds_to_milliseconds(datetime.strptime(start_date, "%Y-%m-%d").timestamp())
        end_date = self._seconds_to_milliseconds(datetime.strptime(end_date, "%Y-%m-%d").timestamp())
        response = self._make_request(
            payload=f"category=spot&symbol={symbol}&interval={interval}&start={start_date}&end={end_date}&limit={limit}")
        self.data = pd.DataFrame(json.loads(response)["result"]["list"]).set_axis(
            ["startTime", "openPrice", "highPrice", "lowPrice", "closePrice", "volume", "turnover"], axis=1)
        self.data['startTime'] = pd.to_datetime(self.data['startTime'].astype(float), unit='ms')
        self.data.set_index('startTime', inplace=True)

        return self


In [9]:
bbdl = BybitDataLoader()
bbdl.load_data(symbol="BTCUSDT", start_date="2024-01-01", end_date="2024-12-11", interval=30)

In [10]:
bbdl.data

,openPrice,highPrice,lowPrice,closePrice,volume,turnover
startTime,,,,,,
2024-12-10 13:00:00,97163.28,97795.99,97119.81,97473.86,918.371366,89553719.6011164
2024-12-10 12:30:00,97259.44,97416.78,96863.19,97163.28,1079.777769,104880606.82005459
2024-12-10 12:00:00,97726.01,97848.06,97250,97259.44,643.09674,62716234.56846574
2024-12-10 11:30:00,97719.94,97924.98,97441.11,97726.01,572.0242,55885871.24795566
2024-12-10 11:00:00,97695.29,97790.32,97520.71,97719.94,469.951823,45887928.08464957
...,...,...,...,...,...,...
2024-11-19 19:30:00,93594.52,93716.98,93028.3,93211.15,592.623868,55372226.43067111
2024-11-19 19:00:00,93635.54,93914.21,93016.1,93594.52,1035.69795,96901876.90048647
2024-11-19 18:30:00,93657.49,93850,93309.35,93635.54,989.798325,92679089.2809949


In [28]:
class FibonacciRetracement:
    """
    Class to perform Fibonacci retracement analysis on historical price data
    """
    FIBONACCI_LEVELS = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]

    def __init__(self):
        self.data = FLAT

    def load_data(self, data: pd.DataFrame) -> FLAT:
        """
        Load and preprocess kline data

        Parameters:
            data: List of dictionaries containing OHLCV data with columns:
                  startTime, openPrice, highPrice, lowPrice, closePrice, volume, turnover
        """
        self.data = data.copy().sort_values('startTime')

    def calculate_fibonacci_levels(self, high: float, low: float) -> Dict[float, float]:
        """
        Calculate Fibonacci retracement levels

        Parameters:
            high: Highest price point
            low: Lowest price point

        Returns:
            Dictionary of Fibonacci levels and their corresponding prices
        """
        return {level: high - ((high - low) * level) for level in self.FIBONACCI_LEVELS}

    def find_swing_points(self, window: int = 10) -> Tuple[List[int], List[int]]:
        """
        Find swing high and low points in the price data

        Parameters:
            window: Number of candles to look before and after for swing point confirmation

        Returns:
            Tuple of lists containing indices of swing highs and lows
        """

        highs = []
        lows = []

        for i in range(window, len(self.data) - window):
            # Check for swing high
            if self.data['highPrice'].iloc[i] == max(self.data['highPrice'].iloc[i - window:i + window + 1]):
                highs.append(i)
            # Check for swing low
            if self.data['lowPrice'].iloc[i] == min(self.data['lowPrice'].iloc[i - window:i + window + 1]):
                lows.append(i)

        return highs, lows

    def identify_trend(self, periods: int = 20) -> str:
        """
        Identify current market trend using SMA

        Parameters:
            periods: Number of periods for moving average calculation

        Returns:
            String indicating trend ('UPTREND' or 'DOWNTREND')
        """
        # simple moving average
        sma = self.data['closePrice'].rolling(window=periods).mean()
        current_price = float(self.data['closePrice'].iloc[-1])
        current_sma = float(sma.iloc[-1])

        return 'UPTREND' if current_price > current_sma else 'DOWNTREND'

    def find_trading_signals(self, window: int = 20) -> pd.DataFrame:
        """
        Find potential trading signals based on Fibonacci retracements

        Parameters:
            window: Number of candles for swing point detection

        Returns:
            DataFrame with signals and Fibonacci levels
        """

        # Create a copy and fill with WAIT signals
        df_signals = self.data.copy()
        df_signals['Signal'] = 'WAIT'
        df_signals['Fib_Level'] = FLAT

        # Find swing points
        highs, lows = self.find_swing_points(window)
        print(f"Found {len(highs)} highs and {len(lows)} lows")

        # Need at least one high and one low
        if len(highs) == 0 or len(lows) == 0:
            print("Not enough swing points found")
            return df_signals

        # Get trend and recent high/low
        trend = self.identify_trend()
        recent_low = float(self.data['lowPrice'].iloc[lows[-1]])
        recent_high = float(self.data['highPrice'].iloc[highs[-1]])

        # Calculate Fibonacci levels
        fibonacci_level_prices = self.calculate_fibonacci_levels(recent_high, recent_low)

        # Check each price point for signals
        for i in range(len(df_signals)):
            price = float(df_signals['closePrice'].iloc[i])

            for level, fib_price in fibonacci_level_prices.items():
                if abs(price - fib_price) / fib_price < 0.01:  # Within 1% of Fibonacci level
                    if trend == 'UPTREND':
                        df_signals.at[df_signals.index[i], 'Signal'] = 'BUY'
                        df_signals.at[df_signals.index[i], 'Fib_Level'] = level
                    elif trend == 'DOWNTREND':
                        df_signals.at[df_signals.index[i], 'Signal'] = 'SELL'
                        df_signals.at[df_signals.index[i], 'Fib_Level'] = level

        return df_signals

    def backtest_strategy(self, initial_capital: float = 10000.0, stop_loss_pct: float = 5.0,
                          take_profit_pct: float = 10.0) -> pd.DataFrame:
        """
        Backtest the Fibonacci retracement strategy (test how it would have performed in the past)

        Parameters:
            initial_capital: Starting capital for the backtest
            stop_loss_pct: Drop in initial price (in %) to trigger a stop loss
            take_profit_pct: Increase in initial price (in %) to trigger a take profit

        Returns:
            DataFrame with backtest results including positions and portfolio values
        """

        df_backtest = self.find_trading_signals()
        df_backtest['Position'] = 'FLAT'
        df_backtest['Portfolio_Value'] = initial_capital

        position = 'FLAT'
        # entry into LONG or SHORT position price
        entry_price = 0.0
        stop_loss_price = 0.0
        take_profit_price = 0.0

        for i in range(1, len(df_backtest)):
            current_signal = df_backtest['Signal'].iloc[i]
            df_backtest.at[df_backtest.index[i], 'Position'] = position

            if position == 'FLAT':
                if current_signal == 'BUY':
                    position = 'LONG'
                    entry_price = float(df_backtest['closePrice'].iloc[i])
                    stop_loss_price = entry_price * (1.0 - stop_loss_pct / 100)
                    take_profit_price = entry_price * (1.0 + take_profit_pct / 100)
                    df_backtest.at[df_backtest.index[i], 'Position'] = position

                elif current_signal == 'SELL':
                    position = 'SHORT'
                    entry_price = float(df_backtest['closePrice'].iloc[i])
                    stop_loss_price = entry_price * (1.0 + stop_loss_pct / 100)
                    take_profit_price = entry_price * (1.0 - take_profit_pct / 100)
                    df_backtest.at[df_backtest.index[i], 'Position'] = position

            elif position == 'LONG':
                current_low = float(df_backtest['lowPrice'].iloc[i])
                current_high = float(df_backtest['highPrice'].iloc[i])

                if current_low < stop_loss_price:
                    # stop holding, time to sell at a loss
                    position = 'FLAT'
                    df_backtest.at[df_backtest.index[i], 'Position'] = 'stop_loss_price'
                elif current_high > take_profit_price:
                    # stop holding, time to sell at a profit
                    position = 'FLAT'
                    df_backtest.at[df_backtest.index[i], 'Position'] = 'take_profit_price'

            elif position == 'SHORT':
                current_low = float(df_backtest['lowPrice'].iloc[i])
                current_high = float(df_backtest['highPrice'].iloc[i])

                if current_high > stop_loss_price:
                    # price gets too high, time to stop borrowing and immediately selling,
                    # time to buy back at a loss
                    position = 'FLAT'
                    df_backtest.at[df_backtest.index[i], 'Position'] = 'stop_loss_price'
                elif current_low < take_profit_price:
                    # price gets too low, time to stop borrowing and immediately selling,
                    # time to buy back for a profit
                    position = 'FLAT'
                    df_backtest.at[df_backtest.index[i], 'Position'] = 'take_profit_price'

            # Update portfolio value
            current_close = float(df_backtest['closePrice'].iloc[i])
            last_value = df_backtest['Portfolio_Value'].iloc[i - 1]

            if position == 'FLAT':
                df_backtest.at[df_backtest.index[i], 'Portfolio_Value'] = last_value
            elif position == 'LONG':
                profit_and_loss = (current_close - entry_price) / entry_price
                df_backtest.at[df_backtest.index[i], 'Portfolio_Value'] = last_value * (1 + profit_and_loss)
            elif position == 'SHORT':
                # note: current_close is the price to buy back
                profit_and_loss = (entry_price - current_close) / entry_price
                df_backtest.at[df_backtest.index[i], 'Portfolio_Value'] = last_value * (1 + profit_and_loss)

        return df_backtest

    def get_current_signal(self) -> Dict:
        """
        Get the current trading signal and additional information

        Returns:
            Dictionary containing current signal, trend, price, and other relevant information
        """
        if self.data is FLAT or len(self.data) == 0:
            raise ValueError("No data available")

        signals = self.find_trading_signals()
        current_signal = signals.iloc[-1]

        return {
            'signal': current_signal['Signal'],
            'fib_level': current_signal['Fib_Level'],
            'trend': self.identify_trend(),
            'current_price': float(self.data['closePrice'].iloc[-1]),
            'timestamp': self.data.index[-1]
        }

    def get_performance_metrics(self, backtest_results: pd.DataFrame) -> Dict:
        """
        Calculate performance metrics from backtest results

        Parameters:
            backtest_results: DataFrame containing backtest results

        Returns:
            Dictionary of performance metrics
        """
        initial_value = float(backtest_results['Portfolio_Value'].iloc[0])
        final_value = float(backtest_results['Portfolio_Value'].iloc[-1])
        total_return = (final_value - initial_value) / initial_value * 100

        trades = backtest_results[backtest_results['Position'].isin(['stop_loss_price', 'take_profit_price'])]
        winning_trades = trades[trades['Position'] == 'take_profit_price']

        metrics = {
            'Total Return (%)': round(total_return, 2),
            'Number of Trades': len(trades),
            'Win Rate (%)': round(len(winning_trades) / len(trades) * 100 if len(trades) > 0 else 0, 2),
            'Average Win (%)': 10.0,  # Fixed due to take profit setting
            'Average Loss (%)': -5.0,  # Fixed due to stop loss setting
            'Final Portfolio Value': f"{round(final_value, 2):,}"
        }

        return metrics

In [29]:
fib = FibonacciRetracement()
fib.load_data(bbdl.data)

In [30]:
signals = fib.find_trading_signals(window=6)

Found 60 highs and 52 lows


In [31]:
signals

,openPrice,highPrice,lowPrice,closePrice,volume,turnover,Signal,Fib_Level
startTime,,,,,,,,
2024-11-19 17:30:00,92821.8,93267.76,92459.99,92842.41,915.640651,85033936.61137009,WAIT,None
2024-11-19 18:00:00,92842.41,93723.47,92704.17,93657.49,1043.428852,97186979.44557171,WAIT,None
2024-11-19 18:30:00,93657.49,93850,93309.35,93635.54,989.798325,92679089.2809949,WAIT,None
2024-11-19 19:00:00,93635.54,93914.21,93016.1,93594.52,1035.69795,96901876.90048647,WAIT,None
2024-11-19 19:30:00,93594.52,93716.98,93028.3,93211.15,592.623868,55372226.43067111,WAIT,None
...,...,...,...,...,...,...,...,...
2024-12-10 11:00:00,97695.29,97790.32,97520.71,97719.94,469.951823,45887928.08464957,BUY,0.5
2024-12-10 11:30:00,97719.94,97924.98,97441.11,97726.01,572.0242,55885871.24795566,BUY,0.5
2024-12-10 12:00:00,97726.01,97848.06,97250,97259.44,643.09674,62716234.56846574,BUY,0.618


In [32]:
results = fib.backtest_strategy()

Found 19 highs and 18 lows


In [33]:
results

,openPrice,highPrice,lowPrice,closePrice,volume,turnover,Signal,Fib_Level,Position,Portfolio_Value
startTime,,,,,,,,,,
2024-11-19 17:30:00,92821.8,93267.76,92459.99,92842.41,915.640651,85033936.61137009,WAIT,None,NONE,1.000000e+04
2024-11-19 18:00:00,92842.41,93723.47,92704.17,93657.49,1043.428852,97186979.44557171,BUY,1,LONG,1.000000e+04
2024-11-19 18:30:00,93657.49,93850,93309.35,93635.54,989.798325,92679089.2809949,BUY,1,LONG,9.997656e+03
2024-11-19 19:00:00,93635.54,93914.21,93016.1,93594.52,1035.69795,96901876.90048647,BUY,1,LONG,9.990934e+03
2024-11-19 19:30:00,93594.52,93716.98,93028.3,93211.15,592.623868,55372226.43067111,BUY,1,LONG,9.943321e+03
...,...,...,...,...,...,...,...,...,...,...
2024-12-10 11:00:00,97695.29,97790.32,97520.71,97719.94,469.951823,45887928.08464957,BUY,0.5,LONG,4.778307e+14
2024-12-10 11:30:00,97719.94,97924.98,97441.11,97726.01,572.0242,55885871.24795566,BUY,0.5,LONG,4.838063e+14
2024-12-10 12:00:00,97726.01,97848.06,97250,97259.44,643.09674,62716234.56846574,BUY,0.618,LONG,4.875179e+14


In [34]:
metrics = fib.get_performance_metrics(results)

In [35]:
metrics

{'Total Return (%)': 4956277133924.72,
 'Number of Trades': 2,
 'Win Rate (%)': 50.0,
 'Average Win (%)': 10.0,
 'Average Loss (%)': -5.0,
 'Final Portfolio Value': '495,627,713,402,471.9'}